# Probabilistic Methods (Discrete)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics
import env

**Anomaly Detection:** Identification of items, events or observations which is significantly different from the remaining data.

- Non-parametric approach
- Frequency or counting based
    - How many time a value of variable (e.g. ip address) shows up
    - More frequent - less likely to be an anomaly
    - less frequent - more likely to be an anomaly
    - Calculate probability 

    
- Conditional probability 
    $$ {P(A|B) = }\frac{\text{P(A U B)}}{\text{P(B)}} $$
    
    
 Examples: 
- How many times we see an ip address in the dataset (count)
- What is probability of ip address showing up in the dataset (ip count / total observations)
- Conditional probability. Given an ip address, what is prob of a particular status(e.g authentication failure)

### Wrangle

The data comes from the webserver logs of the API that we used in the timeseries module. Each row is one request to the server

In [ ]:
url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/logs'
df = pd.read_sql('SELECT * FROM api_access', url)

In [ ]:
df.head(2)

We can isolate the single column as an array and inspect the first couple of elements:

In [ ]:
df['entry'][0]

In [ ]:
df['entry'][1]

Each element is a string and it appears that whitespaces separate key parts of that string. Let's use the `split()` function to chop our string up into an array of those parts:

In [ ]:
df['entry'][0].split()

In [ ]:
# Save the output of the split function on a single element to a variable
parts = df['entry'][0].split()

# Create an dictionary to store labeled components of our log data
output = {}
output['ip'] = parts[0]
output['timestamp'] = parts[3][1:].replace(':', ' ', 1) # We can specify the replace function to only replace the first occurance
output

In [ ]:
pd.Series(output)

Now that we have identified a process for splitting a string up into labeled parts, lets create a function that we can apply to all of our observations, comprehensively labeling each piece of information available:

In [ ]:
# function to deal with parsing one entry in our log data
def parse_log_entry(entry):
    parts = entry.split()
    output = {}
    output['ip'] = parts[0]
    output['timestamp'] = parts[3][1:].replace(':', ' ', 1)
    output['request_method'] = parts[5][1:]
    output['request_path'] = parts[6]
    output['http_version'] = parts[7][:-1]
    output['status_code'] = parts[8]
    output['size'] = int(parts[9])
    output['user_agent'] = ' '.join(parts[11:]).replace('"', '')
    return pd.Series(output)

In [ ]:
df = df.entry.apply(parse_log_entry) # Applying the function to the entire dataframe

In [ ]:
df.head()

Let's add some synthetic anomalies to find in a bit:

In [ ]:
# Synthetic data
new = pd.DataFrame([
    ["95.31.18.119", "21/Apr/2019 10:02:41", "GET", "/api/v1/items/", "HTTP/1.1", '200', 1153005, "python-requests/2.21.0"],
    ["95.31.16.121", "17/Apr/2019 19:36:41", "GET", "/api/v1/sales?page=79/", "HTTP/1.1", '301', 1005, "python-requests/2.21.0"],
    ["97.105.15.120", "18/Apr/2019 19:42:41", "GET", "/api/v1/sales?page=79/", "HTTP/1.1", '301', 2560, "python-requests/2.21.0"],
    ["97.105.19.58", "19/Apr/2019 19:42:41", "GET", "/api/v1/sales?page=79/", "HTTP/1.1", '200', 2056327, "python-requests/2.21.0"],
], columns=df.columns)

df = df.append(new)

In [ ]:
df.tail(5)

It looks like we have some non-unique index elements. We could fix this with a `sort_index()`, but we can also just use a datetime index and those can handle non-unique elements just fine. Let's do that:

In [ ]:
df['size_mb'] = df['size'] / 1024 / 1024 # adding a new column scaled to megabytes for ease of understanding
df.timestamp = pd.to_datetime(df.timestamp)
df = df.set_index('timestamp')

In [ ]:
df.head()

## Detecting Anomalies in Discrete Variables

- **count**: the number of times each unique value appears in the dataset
- **frequencies**: the number of times each unique value appears in the dataset as a percentage of the total; the count divided by the total number of observations

In [ ]:
df.ip.value_counts()

In [ ]:
ip_df = pd.DataFrame(df.ip.value_counts(dropna=False)).reset_index().\
                rename(columns={'index': 'ip', 'ip': 'count'})
ip_df.head()

In [ ]:
df.ip.count()

In [ ]:
# calculate probabity for each ip 

# ip_prob = count for each ip / total count in the dataframe

ip_df2 = pd.DataFrame((df.ip.value_counts(dropna=False))/df.ip.count()).reset_index().\
                rename(columns={'index': 'ip', 'ip': 'proba'})
ip_df2.head()

In [ ]:
11999 / 13978

In [ ]:
# merge the two data frames create above into a single one:
ip_df = ip_df.merge(ip_df2, on='ip')
ip_df.head()

From here we can find how many unique ip addresses there are:

In [ ]:
ip_df.set_index('ip')['count'].sort_values()

In [ ]:
ip_df.set_index('ip')['count'].sort_values().tail(5).plot.barh(figsize=(5,5))

In [ ]:
ip_df.set_index('ip')['count'].sort_values().plot.barh(figsize=(5,5))

**Takeaways**
At this point, we have a list of ip addresses that can be considered anomalous purely based of how infrequently they appear.

### Conditional Probabilities - Intersectionality

- What is probability of a certain status code given an IP address?

HTTP Status Codes

- 200: ok
- 3xx: redirects
- 4xx: client level errors -- the requester did something wrong
- 5xx: server level errors -- the server did something wrong

### Conditional Probability Formula:
                
                prob(A|B) = prob(A & B)/prob(B)

In [ ]:
df.head()

In [ ]:
p_a_and_b = df.groupby(['ip', 'status_code']).size()

In [ ]:
p_a_and_b

In [ ]:
p_a_and_b = df.groupby(['ip', 'status_code']).size()/len(df)

In [ ]:
p_a_and_b

In [ ]:
p_b = df.groupby('ip').size()/len(df)

In [ ]:
p_b

In [ ]:
p_a_and_b / p_b

In [ ]:
# Given a particular ip, what is probability of a certain status code
status_given_ip = (
    df.groupby('ip')\
    .status_code.value_counts(normalize=True)\
    .rename('proba_status_given_ip')\
    .reset_index())
status_given_ip

In [ ]:
status_given_ip[status_given_ip.proba_status_given_ip < 1]

In [ ]:
df[df.ip == '72.181.113.170'].sort_values(by='status_code', ascending=False)

* Cases where the probability is < 100%
* Status codes other than 200

In [ ]:
status_given_ip[status_given_ip.status_code != '200']

#### How do we use this if we are looking to apply assumptions on unseen data?

In [ ]:
df.sort_index().head(1)

In [ ]:
df.sort_index().tail(5)

In [ ]:
# create a training set (removing the fake data I added earlier)
train = df.loc['2019-04-16 19:34:42':'2019-04-17 12:55:14']

In [ ]:
len(train), len(df)

#### Join the probabilities of these events to the original dataframe to detect anomalous events:

In [ ]:
status_given_ip # Essentially a list of the probability of each ip/status code combination

In [ ]:
df = df.reset_index().merge(status_given_ip, on=['ip', 'status_code'], how='left').fillna(value=0).set_index('timestamp')

In [ ]:
df[df.proba_status_given_ip < 0.15]

**All low probability combinations are now easily identifiable with all additional data intact.**